<a href="https://colab.research.google.com/github/kangwonlee/momisp/blob/main/Ch04_SFD.BMD/ex04.002.numpy.simply.supported_m.only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.



`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)



In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비



## 예제 04.002



### 문제에서 주어진 변수<br>Given Parameters



각 구간 길이<br>Length of each section



In [ ]:
L_AB_m = 0.75 * 2
L_BC_m = 0.25 * 2



하중



In [ ]:
M_B_Nm = 3.0



### A, B, C, D 지점의 x 좌표<br>x coorinates of A, B, C, D points
A 점에서 x=0 으로 한다.<br>Let x=0 at A point



In [ ]:
x_A_m = 0
x_B_m = x_A_m + L_AB_m
x_C_m = x_B_m + L_BC_m



### 반력<br>Reaction Force



C점 중심의 모멘트



$$
R_A\left(L_{AB} + L_{BC}\right) + M_B=0\\
$$



$$
R_A = -\frac {M_B}{L_{AB} + L_{BC}}
$$



A 점에서의 반력



In [ ]:
R_A_N = -M_B_Nm / (L_AB_m + L_BC_m)
R_A_N



C 점에서의 반력
$$
R_A + R_C = 0\\
R_C = - R_A
$$



In [ ]:
R_C_N = - R_A_N
R_C_N



### A-B 구간<br>A-B span



In [ ]:
x_AB_m_array = np.linspace(x_A_m, x_B_m, 50+1)



#### 전단력선도<br>Shear Force Diagram



A 지점에서의 하중만 반영하면 된다.



In [ ]:
V_AB_N_array = R_A_N * np.ones_like(x_AB_m_array)



A 지점에서의 반력을 화살표로 표시한다.



In [ ]:
def get_ax(ax=None):
    if ax is None:
        ax = plt.gca()    
    return ax



In [ ]:
def plot_reaction_force_upward(x, length, ax=None):
    ax = get_ax(ax=ax)

    return ax.arrow(
        x, 0, 
        0, length * 0.9, 
        head_width=0.05, head_length=0.1 * abs(length), fc='k', ec='k'
    )



In [ ]:
def format_plot(y_label:str="V(N)"):
    plt.text(x_A_m, 0, 'A')
    plt.text(x_B_m, 0, 'B')
    plt.text(x_C_m, 0, 'C')

    xlim = plt.xlim()
    plt.xlim(xmax=max(x_C_m, max(xlim)))
    plt.xlabel('x(m)')
    plt.ylabel(y_label)
    plt.grid(True)



In [ ]:
plot_reaction_force_upward(x=x_A_m, length=R_A_N)

plt.fill_between(x_AB_m_array, V_AB_N_array)

format_plot()
plt.show()



#### 굽힘모멘트선도<br>Bending Moment Diagram



In [ ]:
M_AB_Nm_array = R_A_N * x_AB_m_array



In [ ]:
plt.fill_between(x_AB_m_array, M_AB_Nm_array)

format_plot("M(Nm)")
plt.show()



### B-C 구간<br>B-C span



In [ ]:
x_BC_m_array = np.linspace(x_B_m, x_C_m, 25+1)



#### 전단력선도<br>Shear Force Diagram



계속해서 A 지점에서의 반력만 감안한다.



In [ ]:
V_BC_N_array = R_A_N * np.ones_like(x_BC_m_array)



In [ ]:
def plot_reaction_force_downward(x, length, ax=None):
    ax = get_ax(ax=ax)

    return ax.arrow(
        x, -length, 
        0, length * 0.9, 
        head_width=0.05, head_length=0.1 * abs(length), fc='k', ec='k'
    )



In [ ]:
def plot_SFD(ax=None):
    ax = get_ax(ax=ax)

    # A 지점에서의 반력을 화살표로 표시한다.
    plot_reaction_force_upward(x=x_A_m, length=R_A_N, ax=ax)
    # C 지점에서의 반력을 화살표로 표시한다.
    plot_reaction_force_downward(x=x_C_m, length=R_C_N, ax=ax)

    plt.fill_between(x_AB_m_array, V_AB_N_array)
    plt.fill_between(x_BC_m_array, V_BC_N_array)

    format_plot()



In [ ]:
plt.clf()
plot_SFD()
plt.show()



#### 굽힘모멘트선도<br>Bending Moment Diagram



B 지점의 모멘트의 영향을 반영한다.



In [ ]:
M_BC_Nm_array = R_A_N * x_BC_m_array + M_B_Nm



In [ ]:
def plot_moment_load(x, bmd_x, load, ax=None):
    ax = get_ax(ax=ax)

    return ax.arrow(
        x, bmd_x, 
        0, load * 0.9, 
        head_width=0.05, head_length=0.1 * abs(load), fc='k', ec='k'
    )



In [ ]:
def plot_BMD(ax=None):
    ax = get_ax(ax=ax)
    
    plot_moment_load(x_B_m, R_A_N * x_B_m, M_B_Nm)

    plt.fill_between(x_AB_m_array, M_AB_Nm_array)
    plt.fill_between(x_BC_m_array, M_BC_Nm_array)

    format_plot("M(Nm)")



In [ ]:
plt.clf()
plot_BMD()
plt.show()



아래위로 나란히 그려 본다



In [ ]:
ax = plt.subplot(2, 1, 1)
plot_SFD(ax)

ax = plt.subplot(2, 1, 2)
plot_BMD(ax)

plt.show()

